In [70]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np

%load_ext autoreload
%autoreload 2

cols = ['Record Id',	
'Created Time',
'Created Date',	
'First Name',	
'Last Name',	
'Email',	
'PHONE',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Nationality',
'Booking Status', 
'BookRef',
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
path = r'C:\Users\fajar\Documents\Python\Data\karma beach 2.xlsx'
# zoho_df = pd.read_csv(path, dtype='string')
zoho_df = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

zoho_df['Mobile'] = 'empty'
zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Do you live in Bali?,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Regions,Mobile
0,zcrm_3584690000006081163,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,raux.sophie@gmail.com,33660458813,empty,United States,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
1,zcrm_3584690000006081164,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,adennis2@dow.com,empty,empty,Australia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
2,zcrm_3584690000006081167,2019-04-02 03:27:08,2015-03-11 00:00:00,Camilla,Frank,camilla@camilla.com.au,61403133367,empty,Australia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
3,zcrm_3584690000006081168,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,mathew.lamb@marinabaysands.com,6592332898,empty,Hong Kong,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,Other,empty
4,zcrm_3584690000006081169,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,eliana.gamboa@fco.gov.uk,empty,empty,Indonesia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17951,empty,empty,2023-12-20 00:00:00,Marcel,Berthelette,mberthelette13@gmail.com,60 122207911,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17952,empty,empty,2023-12-26 00:00:00,May Sari,Hendrawati,maysari.hendrawati@gmail.com,60 172808054,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17953,empty,empty,2023-12-26 00:00:00,Erwin,Horianto,erwinhorianto@yahoo.com,62 85103144444,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17954,empty,empty,2023-12-26 00:00:00,Haryo,Armono,haryodwito@gmail.com,62 81944937203,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty


In [72]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


# ====================== Awal Filter Umum ====================== 

In [73]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

'Email'

In [74]:
count_deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

# ====================== Akhir Filter Umum ====================== 

In [75]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Do you live in Bali?,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Regions,Mobile
49,zcrm_3584690000161305878,2020-01-27 15:27:57,empty,Michael,Southwell,mpsouthwell@hotmail.com,61405264923,empty,Australia,False,Karma Resorts,Karma Beach,Web Enquiry,Form Contact Us,SEAP,empty
310,zcrm_3584690000171661098,2020-03-05 10:28:45,empty,Phoebe,Middleton,phoebemid@hotmail.co.uk,447971076533,empty,United Kingdom,False,Karma Resorts,Karma Beach,Web Enquiry,Purple Wifi,UK/Europe,empty
668,zcrm_3584690000472465294,2023-12-11 17:55:38,empty,Dita,Ang,tatatititutusebelas@yahoo.com,6287881145044,empty,empty,False,Karma Resorts,Karma Beach,Other,Reservation di Mare Restaurant & Karma Beach ...,SEAP,empty
824,zcrm_3584690000472465708,2023-12-11 17:55:40,empty,Valerya,Babarika,dashasafonkina@yandex.ru,6287741857355,empty,empty,False,Karma Resorts,Karma Beach,Other,Reservation di Mare Restaurant & Karma Beach ...,SEAP,empty
858,zcrm_3584690000472465742,2023-12-11 17:55:40,empty,Pernille,Bannan,james.bannan@coeli.com,4553809879,empty,empty,False,Karma Resorts,Karma Beach,Other,Reservation di Mare Restaurant & Karma Beach ...,SEAP,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17217,empty,empty,2023-11-05 00:00:00,Frank,Waer´op,frankwaerlop@gmail.com,32498414182,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17312,empty,empty,2024-03-19 00:00:00,Marco,Weiss,marco_weiss@hotmail.it,393482400741,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17407,empty,empty,2024-03-09 00:00:00,Vanessa,De vries,vanesdevries@hotmail.com,6281558885053,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17580,empty,empty,2023-12-22 00:00:00,Torben,Wick,torben.wick@outlook.com,61491103687,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty


In [76]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Do you live in Bali?,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Regions,Mobile
0,zcrm_3584690000006081163,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,raux.sophie@gmail.com,33660458813,empty,United States,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
1,zcrm_3584690000006081164,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,adennis2@dow.com,empty,empty,Australia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
2,zcrm_3584690000006081167,2019-04-02 03:27:08,2015-03-11 00:00:00,Camilla,Frank,camilla@camilla.com.au,61403133367,empty,Australia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
3,zcrm_3584690000006081168,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,mathew.lamb@marinabaysands.com,6592332898,empty,Hong Kong,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,Other,empty
4,zcrm_3584690000006081169,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,eliana.gamboa@fco.gov.uk,empty,empty,Indonesia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17761,empty,empty,2023-12-20 00:00:00,Marcel,Berthelette,mberthelette13@gmail.com,60122207911,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17762,empty,empty,2023-12-26 00:00:00,May Sari,Hendrawati,maysari.hendrawati@gmail.com,60172808054,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17763,empty,empty,2023-12-26 00:00:00,Erwin,Horianto,erwinhorianto@yahoo.com,6285103144444,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17764,empty,empty,2023-12-26 00:00:00,Haryo,Armono,haryodwito@gmail.com,6281944937203,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty


In [77]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Do you live in Bali?,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Regions,Mobile
4404,zcrm_3584690000472559255,2023-12-11 17:56:10,empty,Winnie,Test,longlonglongyan13@163.com,8618810614235,empty,empty,False,Karma Resorts,Karma Beach,Other,Reservation di Mare Restaurant & Karma Beach ...,SEAP,empty
5508,zcrm_3584690000513571316,2024-10-29 11:46:49,empty,Tes,Test,fbadmin@karmakandara.com,6281138107130,empty,empty,False,Karma Resorts,Karma Beach,Other,Reservation di Mare Restaurant & Karma Beach -...,empty,empty
7295,empty,empty,2024-09-25 00:00:00,Kitty,Marsh,test@mail.com,971555555532,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
12171,empty,empty,2024-06-19 00:00:00,Stephen,Test,travy@gmail.com,622526266089,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
12907,empty,empty,2024-06-02 00:00:00,it,test,it@karma.com,6283288542287,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty


In [78]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [79]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [80]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [81]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Do you live in Bali?,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Regions,Mobile
166,zcrm_3584690000164583275,2020-02-10 07:00:49,empty,Yagoda,Kiparizov,yagoda.kiparizov@rams.com.au,610416253238,empty,Australia,False,Karma Resorts,Karma Beach,Web Enquiry,Form Contact Us,SEAP,empty
1429,zcrm_3584690000472553098,2023-12-11 17:55:46,empty,John,Curtis,victoriaetournaud@gmail.com,62423700438,empty,empty,False,Karma Resorts,Karma Beach,Other,Reservation di Mare Restaurant & Karma Beach ...,SEAP,empty
1823,zcrm_3584690000472553763,2023-12-11 17:55:47,empty,Paul,Frankfurt,i5y7bpnyas@m.expediapartnercentral.com,62361701010,empty,empty,False,Karma Resorts,Karma Beach,Other,Reservation di Mare Restaurant & Karma Beach ...,SEAP,empty
6582,empty,empty,2024-07-28 00:00:00,Will,Bezak,asiaptour@gmail.com,6281219967403,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
15611,empty,empty,2024-04-01 00:00:00,Natalia,Lyu,jjtour@mail.ru,79263574560,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty


In [82]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Do you live in Bali?,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Regions,Mobile
0,zcrm_3584690000006081163,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,raux.sophie@gmail.com,33660458813,empty,United States,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
1,zcrm_3584690000006081164,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,adennis2@dow.com,empty,empty,Australia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
2,zcrm_3584690000006081167,2019-04-02 03:27:08,2015-03-11 00:00:00,Camilla,Frank,camilla@camilla.com.au,61403133367,empty,Australia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
3,zcrm_3584690000006081168,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,mathew.lamb@marinabaysands.com,6592332898,empty,Hong Kong,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,Other,empty
4,zcrm_3584690000006081169,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,eliana.gamboa@fco.gov.uk,empty,empty,Indonesia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17751,empty,empty,2023-12-20 00:00:00,Marcel,Berthelette,mberthelette13@gmail.com,60122207911,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17752,empty,empty,2023-12-26 00:00:00,May Sari,Hendrawati,maysari.hendrawati@gmail.com,60172808054,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17753,empty,empty,2023-12-26 00:00:00,Erwin,Horianto,erwinhorianto@yahoo.com,6285103144444,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17754,empty,empty,2023-12-26 00:00:00,Haryo,Armono,haryodwito@gmail.com,6281944937203,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty


In [83]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Do you live in Bali?,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Regions,Mobile
552,zcrm_3584690000472465177,2023-12-11 17:55:38,empty,Clinton,Albuquerque,clinton.albuquerque@karmagroup.com,628123868046,empty,empty,False,Karma Resorts,Karma Beach,Other,Reservation di Mare Restaurant & Karma Beach ...,SEAP,empty
5030,zcrm_3584690000513405303,2024-10-29 11:46:44,empty,Rai,Artawan,rai.artawan@karmakandara.com,6281338016603,empty,empty,False,Karma Resorts,Karma Beach,Other,Reservation di Mare Restaurant & Karma Beach -...,empty,empty
7940,empty,empty,2024-09-04 00:00:00,Adis,It,adi.sumardiyasa@karmagroup.com,6282184938293,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
11219,empty,empty,2024-04-26 00:00:00,arya,wijaya,it.kk@karmaresorts.com,6285739200959,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
12099,empty,empty,2024-06-21 00:00:00,George,robertson,george.robertson@karmagroup.com,6277758764,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
13528,empty,empty,2024-05-11 00:00:00,Dyah,Retnosari,dyah.retnosari@karmagroup.com,6281999257060,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty


In [84]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,State,Country,Do you live in Bali?,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Regions,Mobile
0,zcrm_3584690000006081163,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,raux.sophie@gmail.com,33660458813,empty,United States,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
1,zcrm_3584690000006081164,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,adennis2@dow.com,empty,empty,Australia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
2,zcrm_3584690000006081167,2019-04-02 03:27:08,2015-03-11 00:00:00,Camilla,Frank,camilla@camilla.com.au,61403133367,empty,Australia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
3,zcrm_3584690000006081168,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,mathew.lamb@marinabaysands.com,6592332898,empty,Hong Kong,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,Other,empty
4,zcrm_3584690000006081169,2019-04-02 03:27:08,2015-05-12 00:00:00,-,-,eliana.gamboa@fco.gov.uk,empty,empty,Indonesia,False,Karma Resorts,Karma Beach,Referral,Nammos Beach Club Business Card 2011,SEAP,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17745,empty,empty,2023-12-20 00:00:00,Marcel,Berthelette,mberthelette13@gmail.com,60122207911,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17746,empty,empty,2023-12-26 00:00:00,May Sari,Hendrawati,maysari.hendrawati@gmail.com,60172808054,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17747,empty,empty,2023-12-26 00:00:00,Erwin,Horianto,erwinhorianto@yahoo.com,6285103144444,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty
17748,empty,empty,2023-12-26 00:00:00,Haryo,Armono,haryodwito@gmail.com,6281944937203,empty,empty,False,Timeshare Marketing,Other Bali DB,Other,Purple Wifi,Indonesia,empty


In [85]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [86]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [87]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [88]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

206

In [89]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 17956
Jumlah Format Email Salah : 0
Jumlah Setelah Clean Format Email Salah : 17956
Jumlah Duplicate (Phone, Mobile dan Email) : 190
Jumlah Setelah Clean Duplicate : 17766
Jumlah Lead Test : 5
Jumlah Setelah Clean Lead Test : 17761
Jumlah Email OTA : 5
Jumlah Setelah Clean Email OTA : 17756
Jumlah Email Karma : 6
Jumlah Setelah Clean Email Karma : 17750
Jumlah Wash Member : 1302
Jumlah Wash Booking : 33
Jumlah Setelah Wash Member : 16415
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 0
Jumlah Setelah Wash Phone Email Blank : 16415
Contact Type
Jumlah Email Only : 17
Jumlah Phone Only : 1
Jumlah Email dan Phone : 16397


In [90]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 17956
Jumlah Wash Dedup : 206
Jumlah Wash Member : 1302
Jumlah Wash Booking : 33
Jumlah Data Akhir : 16415


In [91]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['First Name'] = zoho_df['First Name'].str.title()
zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

# zoho_df.to_csv(r'C:\Users\fajar\Documents\Python\Data\karma beach cleand.csv', index=False)
zoho_df.to_excel(r'C:\Users\fajar\Documents\Python\Data\karma beach clean 2.xlsx', index=False)